### reference link
https://github.com/catSirup/naver_kin_crawling/blob/master/project.py

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
import time
import os

import pandas as pd
from random import *
# from multiprocessing import Pool, cpu_count
# import multiprocessing as mp


import warnings
warnings.filterwarnings(action='ignore') 


In [14]:
#현재 내 위치
currentPath=os.getcwd()
print(currentPath)

/Users/heojiyeon/크롤러


# 로그인 

In [15]:
options = Options()
options.add_argument('--headless') 

#driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
driver = webdriver.Chrome(ChromeDriverManager().install())

# 네이버 로그인 화면 이동
login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
driver.get(login_url)
driver.implicitly_wait(10)


# 아이디& 비밀번호 입력
my_id = 'id' # "id" 대신에 자신의 네이버 아이디가 들어가야 합니다
my_pw = 'pw' # "password" 대신에 자신의 네이버 비밀번호가 들어가야 합니다


# 로그인 id, pw 입력
# 네이버에 로그인 할 경우 'send_keys()' 함수가 아니라 'execute_script()' 함수를 사용
# 여기 코드부터는 건드리지 않습니다!!
driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
time.sleep(1)


# '로그인' 버튼 클릭
driver.find_element_by_id('log.login').click()
time.sleep(1)




====== WebDriver manager ======
Current google-chrome version is 111.0.5563
Get LATEST chromedriver version for 111.0.5563 google-chrome
Driver [/Users/heojiyeon/.wdm/drivers/chromedriver/mac64/111.0.5563.64/chromedriver] found in cache


# 키워드 URL 수집

In [16]:
# and로 바꾸기: %2B
def get_keyword(master_keywords, K):
    text=master_keywords.split(" ")
    result=""
    for i in range(len(text)):
        a=text[i]+"%2B"+K 
        result += a+","
    return result[:-1]

# 정렬 방식 선택
# 1: 추천순
# 2: 최신순
# 기타: 정확도 순
def sort_kind(index):
    # 추천
    if index == 1:
        return 'vcount'
    # 최신순
    elif index == 2:
        return 'date'
    # 정확도
    else:
        return 'none'


#마스터 키워드
# '' 안에 "" 또 넣어주면 정확하게 일치하는 단어 찾음

keyword='"신혼부부" "장보기"' # 공백으로 분리해주세요

#절대 키워드
KEY='"강아지"' 

    
#결합 키워드
master_keyword= get_keyword(keyword, KEY)

#각각 키워드 조합 검색하기
for i in master_keyword.split(","):
    driver.get('https://kin.naver.com/search/list.nhn?query=' + i)
    time.sleep(uniform(0.1, 1.0))

    page_index = 1

    # 크롤링 시작 일자
    f = '2022.02.25' #형식: 2022.12.01

    # 크롤링 종료 일자
    t = '2023.02.28'
    period_txt = "&period=" + f + ".%7C" + t + "."

    #추천순:1, 최신순:2 정확도:3
    num=2
    _sort_kind = sort_kind(int(num))

    date = str(datetime.now()).replace('.', '_')
    date = date.replace(' ', '_')

    # URL 저장
    path='/Users/heojiyeon/크롤러/result'
    f = open(f"{path}" + "/url_list" + "_" + keyword.replace(' ', '+') + "_" + date + ".txt", 'w')
    page_url = []

    while True:
        time.sleep(uniform(0.01, 1.0))
        driver.get('https://kin.naver.com/search/list.nhn?' + "&sort=" + _sort_kind + '&query=' + i+ period_txt + "&section=kin" + "&page=" + str(page_index))
        html = driver.page_source
        soup = bs(html, 'html.parser')

        tags = soup.find_all('a', class_="_nclicks:kin.txt _searchListTitleAnchor")
        for tag in tags:
            url = str(tag).split(' ')[3]
            url = url.replace('href=', "")
            url = url.replace('"', "")
            url = url.replace('amp;', '')
            page_url.append(url)
            f.write(url + "\n")

        post_number = driver.find_element_by_class_name('number').text
        post_number = str(post_number).replace("(", "")
        post_number = str(post_number).replace(")", "")
    
        current_number = post_number.split('/')[0].split('-')[1]
        current_number = current_number.replace(',', '')
        total_number = post_number.split('/')[1]
        total_number = total_number.replace(',', '')

        if int(current_number) == int(total_number):
            break
        else:
            page_index += 1
        
print(len(page_url))

24


# 제목, 질문, 답변, 날짜 

In [17]:
filename = 'result/' + keyword.replace(' ', '.') + "_" + date + "_crawling_result.xlsx"

Content_w=[] #작성자의 제목+질문 저장할 빈리스트
Date_w=[]

Content_a=[] #답변 저장 빈리스트
Date_a=[]
    
#크롤링 페이지 카운트
count=0
for i in page_url:
    driver.get(i)
    
    #글쓴이 
    #제목
    title = driver.find_element_by_class_name('title').text

    #질문
    try:
        question_txt = driver.find_element_by_class_name('c-heading__content').text
        
    except:
        question_txt = ""
    Content_w.append(question_txt+' '+title)
    
    #날짜
    date_list=driver.find_element_by_class_name("c-userinfo__info").text.split('\n')
    
    if len(date_list) >2:
        date=date_list[2] 
    else:
        date=date_list[1] 
        
    Date_w.append(date)

    
    # 답변 리스트 
    # 글쓴이의 제목&질문 하나에 다수의 질문이 달림
    answer_list = driver.find_elements_by_class_name("se-main-container")
    date_list=driver.find_elements_by_class_name("c-heading-answer__content-date")
    
    for i in range(len(answer_list)):
        t=answer_list[i].text
        d=date_list[i].text
        Content_a.append(t) 
        Date_a.append(d)

     
    
    #댓글 버튼 전부 누르기
    # 클래스 이름이 button_compose인 모든 버튼을 찾음
    elements = driver.find_elements_by_class_name('button_compose')
    comment_buttons = [elem for elem in elements if ' _commentBtn' in elem.get_attribute('class')]

    for button in comment_buttons:
        button.click() # 버튼 클릭
        time.sleep(uniform(0.1, 0.3))
        
        
        
    #### 여기서부터 너무 느림... 아직 해결 중...
    try:
        #대댓글 내용 
        rep_list=driver.find_elements_by_class_name('c-opinion__list-text')    
        rep_date=driver.find_elements_by_class_name("c-opinion__list-date")
        for i,j in zip(rep_list,rep_date):
            rep_date=j.text
            rep_text=i.text
            Content_a.append(rep_text)
            Date_a.append(rep_date)
    except:
        pass
    
    try:
        #추가질문
        # additionalQnaList 클래스를 가진 div 요소를 찾습니다.
        additional_qnas = driver.find_elements_by_class_name("addition-qna__item")

        # additional_qnas에서 각각의 div 요소에서 추가질문과 추가답변 텍스트를 추출합니다.
        for additional_qna in additional_qnas:
        
            # 추가질문인 경우
            if "addQuestion.svg" in additional_qna.get_attribute("innerHTML"):
                add_q_text = additional_qna.find_element_by_class_name("addition-qna__info-text").text
                add_q_date= additional_qna.find_element_by_class_name("addition-qna__info-date").text
                Content_a.append(add_q_text)
                Date_a.append(add_q_date)

            # 추가답변인 경우
            elif "addAnswer.svg" in additional_qna.get_attribute("innerHTML"):
                add_a_text = additional_qna.find_element_by_class_name("addition-qna__info-text").text
                add_a_date= additional_qna.find_element_by_class_name("addition-qna__info-date").text
                Content_a.append(add_q_text)
                Date_a.append(add_q_date)
                    
    except:
        pass
        
    
    count += 1
    print(count)


# 크롬 창 닫기
#driver.quit()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [18]:
print(len(Content_w)); len(Date_w)
print(len(Content_a)); len(Date_a)

24
87


87

In [19]:
# 데이터 합치기
total_w=pd.DataFrame({'text':Content_w, 'date':Date_w})
total_a=pd.DataFrame({'text':Content_w, 'date':Date_w})
result=pd.concat([total_w, total_a])
result

,text,date
0,저희 강아지는 이번에 3살 된 여아 말티즈인데 오늘 산책을 나가서 얘보다 3배는 큰...,2023.02.12
1,2시간 이내 거리로 추천부탁드립니다! 강아지 동반 가능한갈로요 인천과 가까운 가족탕...,2023.02.04
2,저가 바람을펴서 와이프가 상간녀소송을하였습니다\n상간소송은 상간녀가 위자료 지급하여...,2023.02.03
3,가족끼리만 자는 펜션으로 알려주세여 이층집에 수영장 있어야해용 인원 5명 강아지 1...,2023.02.01
4,20대 안에 꼭 해보고 싶은\n버킷 리스트인데 다들 몇번몇번\n해보셨나요?\n\n0...,2023.01.18
5,20대 안에 꼭 해보고 싶은\n버킷 리스트인데 다들 몇번몇번\n해보셨나요?\n\n0...,2023.01.18
6,20대 안에 꼭 해보고 싶은\n버킷 리스트인데 다들 몇번몇번\n해보셨나요?\n\n0...,2023.01.18
7,드디어 다음주면 제주도로 갑니다~\n강아지식구가 생기고 처음가는 여행이라\n신경쓸게...,2022.11.11
8,어렸을때 읽었던 동화책을 찾고 싶어요ㅜㅜ\n외국 동화로 기억하는데\n\n쌍둥이인지 ...,2022.10.28
9,저희 강아지는 이번에 3살 된 여아 말티즈인데 오늘 산책을 나가서 얘보다 3배는 큰...,2022.10.15


In [20]:
result.to_excel(filename)

# 잘 저장됐는지 확인

In [21]:
import pandas as pd
al=pd.read_excel(filename)
al

,Unnamed: 0,text,date
0,0,저희 강아지는 이번에 3살 된 여아 말티즈인데 오늘 산책을 나가서 얘보다 3배는 큰...,2023.02.12
1,1,2시간 이내 거리로 추천부탁드립니다! 강아지 동반 가능한갈로요 인천과 가까운 가족탕...,2023.02.04
2,2,저가 바람을펴서 와이프가 상간녀소송을하였습니다\n상간소송은 상간녀가 위자료 지급하여...,2023.02.03
3,3,가족끼리만 자는 펜션으로 알려주세여 이층집에 수영장 있어야해용 인원 5명 강아지 1...,2023.02.01
4,4,20대 안에 꼭 해보고 싶은\n버킷 리스트인데 다들 몇번몇번\n해보셨나요?\n\n0...,2023.01.18
5,5,20대 안에 꼭 해보고 싶은\n버킷 리스트인데 다들 몇번몇번\n해보셨나요?\n\n0...,2023.01.18
6,6,20대 안에 꼭 해보고 싶은\n버킷 리스트인데 다들 몇번몇번\n해보셨나요?\n\n0...,2023.01.18
7,7,드디어 다음주면 제주도로 갑니다~\n강아지식구가 생기고 처음가는 여행이라\n신경쓸게...,2022.11.11
8,8,어렸을때 읽었던 동화책을 찾고 싶어요ㅜㅜ\n외국 동화로 기억하는데\n\n쌍둥이인지 ...,2022.10.28
9,9,저희 강아지는 이번에 3살 된 여아 말티즈인데 오늘 산책을 나가서 얘보다 3배는 큰...,2022.10.15
